In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip -q install transformers
!pip -q install datasets
!pip -q install simpletransformers
!pip -q install pythainlp

     |████████████████████████████████| 4.4 MB 7.3 MB/s 
     |████████████████████████████████| 596 kB 82.1 MB/s 
     |████████████████████████████████| 101 kB 13.9 MB/s 
     |████████████████████████████████| 6.6 MB 50.1 MB/s 
     |████████████████████████████████| 362 kB 9.6 MB/s 
     |████████████████████████████████| 212 kB 81.7 MB/s 
     |████████████████████████████████| 1.1 MB 66.0 MB/s 
     |████████████████████████████████| 140 kB 92.3 MB/s 
     |████████████████████████████████| 127 kB 80.9 MB/s 
     |████████████████████████████████| 94 kB 1.1 MB/s 
     |████████████████████████████████| 144 kB 96.2 MB/s 
     |████████████████████████████████| 271 kB 85.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from datasets import load_dataset
thaiqa_ = load_dataset('thaiqa_squad')

Reusing dataset thaiqa_squad (/root/.cache/huggingface/datasets/thaiqa_squad/thaiqa_squad/1.0.0/fce14864b511d48464540780f328f4b415746b63f2fd934ad0b06c3eead7787b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!wget https://archive.org/download/iapp_thaiqa/iapp_thaiqa.zip
!unzip iapp_thaiqa.zip


In [ ]:
%load_ext autoreload
%autoreload 2

import pythainlp, transformers
pythainlp.__version__, transformers.__version__ #fix pythainlp to stabilize word tokenization for metrics
import collections
import logging
import pprint
import re
from tqdm.auto import tqdm

import numpy as np
import torch

#datasets
from datasets import (
    load_dataset, 
    load_metric, 
    concatenate_datasets,
    load_from_disk,
)


In [ ]:
thaiqa_

DatasetDict({
    train: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 74
    })
})

In [ ]:
import pandas as pd
q1 = pd.DataFrame(thaiqa_["train"])
q1 = q1.loc[:,['question_id', 'article_id', 'context', 'question', 'answers']]
q2 = pd.DataFrame(thaiqa_["validation"])
q2 = q2.loc[:,['question_id', 'article_id', 'context', 'question', 'answers']]


In [ ]:
from lxml import etree
parser = etree.XMLParser(recover=True)
tree = etree.fromstring(q1["context"][0], parser=parser)
print(tree.tag)
print(tree.attrib)
print(tree.text)

doc
{'id': '115035', 'url': 'https://th.wikipedia.org/wiki?curid=115035', 'title': 'เบนจี้'}
เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู้ ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบท และกำกับโดย โจ แคมป์ ในช่วงทศวรรษ 1970 ถึง 1980 ภาพยนตร์เรื่องแรกในชุด ใช้ชื่อเรื่องว่า เบนจี้ เช่นเดียวกับตัวละคร ถ่ายทำที่เมืองดัลลัส รัฐเทกซัส ฉายครั้งแรกในปี พ.ศ. 2517 ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์ และได้รางวัลลูกโลกทองคำ สาขาเพลงประกอบยอดเยี่ยม จากเพลง Benji's Theme (I Feel Love) ร้องโดย ชาร์ลี ริช หมาที่แสดงเป็นเบนจี้ตัวแรก ชื่อว่า ฮิกกิ้นส์ (พ.ศ. 2502 - พ.ศ. 2518) มีอายุถึง 15 ปีแล้วในขณะแสดง หลังจากภาพยนตร์ออกฉายได้ไม่นาน มันก็ตายในปี พ.ศ. 2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์. - พ.ศ. 2517, Benji (ภาพยนตร์) - พ.ศ. 2520, For the Love of Benji (ภาพยนตร์) - พ.ศ. 2521, Benji's Very Own Christmas Story (ภาพยนตร์โทรทัศน์) - พ.ศ. 2523, Oh Heavenly Dog (ภาพยนตร์) - พ.ศ. 2523, Benji at Work (ภาพยนตร์โทรทัศน์) - พ.ศ. 2524, Benji Takes a Dive at Marineland (ภาพยนตร์โทรทัศน์) - พ.ศ. 2526, Benji

In [ ]:
def extract_text(body):
  tree = etree.fromstring(body, parser=parser)
  return tree.text

In [ ]:
q1["context_text"] = q1["context"].apply(extract_text)
q2["context_text"] = q2["context"].apply(extract_text)

In [ ]:
# qa_train.head()

,question_id,article_id,title,context,question,answers
0,01KZTrxgvC5mOovXFMPJ_000,01KZTrxgvC5mOovXFMPJ,พัทธ์ธีรา ศรุติพงศ์โภคิน,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร,"{'answer_end': [50], 'answer_start': [31], 'te..."
1,01KZTrxgvC5mOovXFMPJ_001,01KZTrxgvC5mOovXFMPJ,พัทธ์ธีรา ศรุติพงศ์โภคิน,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร,"{'answer_end': [70], 'answer_start': [68], 'te..."
2,01KZTrxgvC5mOovXFMPJ_002,01KZTrxgvC5mOovXFMPJ,พัทธ์ธีรา ศรุติพงศ์โภคิน,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร,"{'answer_end': [92], 'answer_start': [75], 'te..."
3,01KZTrxgvC5mOovXFMPJ_003,01KZTrxgvC5mOovXFMPJ,พัทธ์ธีรา ศรุติพงศ์โภคิน,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร,"{'answer_end': [155], 'answer_start': [139], '..."
4,01asKzWKkocCMjZAITfL_001,01asKzWKkocCMjZAITfL,คลีโอพัตรา,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,บิดาของคลีโอพัตราเป็นใคร,"{'answer_end': [339], 'answer_start': [319], '..."


In [ ]:
from pythainlp.tokenize import word_tokenize
q1["context_text_wordsegged"] = q1["context"].apply(lambda x: ' '.join(word_tokenize(x)))
q2["context_text_wordsegged"] = q2["context"].apply(lambda x: ' '.join(word_tokenize(x)))


In [ ]:
q1["question_wordsegged"] = q1["question"].apply(lambda x: ' '.join(word_tokenize(x)))
q2["question_wordsegged"] = q2["question"].apply(lambda x: ' '.join(word_tokenize(x)))


In [ ]:
q1["answers_wordsegged"] =  q1["answers"].apply(lambda x: ' '.join(word_tokenize(x["text"][0])))
q2["answers_wordsegged"] =  q2["answers"].apply(lambda x: ' '.join(word_tokenize(x["text"][0])))


In [ ]:
answer_starts = []
for idx, row in q1.iterrows():
  try:
    answer_start = row["context_text_wordsegged"].index(row["answers_wordsegged"])
  except:
    answer_start = -1
  answer_starts.append(answer_start)

In [ ]:
q1["answer_starts"] = answer_starts

In [ ]:
answer_starts = []
for idx, row in q2.iterrows():
  try:
    answer_start = row["context_text_wordsegged"].index(row["answers_wordsegged"])
  except:
    answer_start = -1
  answer_starts.append(answer_start)

In [ ]:
q2["answer_starts"] = answer_starts

In [ ]:
train_data = []
for idx, row in q1.iterrows():
  doc = {}
  doc["context"] = row["context_text_wordsegged"]
  doc["qas"] = [{"id":idx, "is_impossible":row["answer_starts"]<0,"question":row["question_wordsegged"],"answers":[{"text":row["answers_wordsegged"], "answer_start":row["answer_starts"]}]}]
  train_data.append(doc)

In [ ]:
test_data = []
for idx, row in q2.iterrows():
  doc = {}
  doc["context"] = row["context_text_wordsegged"]
  doc["qas"] = [{"id":idx, "is_impossible":row["answer_starts"]<0,"question":row["question_wordsegged"],"answers":[{"text":row["answers_wordsegged"], "answer_start":row["answer_starts"]}]}]
  test_data.append(doc)

In [ ]:
len(test_data)

742

In [ ]:
len(train_data)

9723

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel,QuestionAnsweringArgs


# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 128
model_args.evaluate_during_training = True
model_args.num_train_epochs = 1
model_args.overwrite_output_dir = True

# model_args = {
#     "reprocess_input_data": True,
#     'train_batch_size':32,
#     "use_early_stopping": True,
#     "early_stopping_delta": 0.01,
#     "early_stopping_metric": "mcc",
#     "early_stopping_metric_minimize": False,
#     "early_stopping_patience": 5,
#     "evaluate_during_training_steps": 500,
#     "fp16": False,
#     "num_train_epochs":3,
#     "overwrite_output_dir":True
# }

Bert = QuestionAnsweringModel(
    "longformer", "valhalla/longformer-base-4096-finetuned-squadv1", args=model_args
)

# Train the model
Bert.train_model(train_data, eval_data=test_data)

# Evaluate the model
result, texts = Bert.eval_model(train_data)

add example index and unique id: 100%|██████████| 9723/9723 [00:00<00:00, 146287.74it/s]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/915 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
result, texts = Bert.eval_model(train_data)

add example index and unique id: 100%|██████████| 14465/14465 [00:00<00:00, 287372.27it/s]


Running Evaluation:   0%|          | 0/9511 [00:00<?, ?it/s]

In [ ]:
result

{'correct': 2577,
 'eval_loss': -8.6109061218589,
 'incorrect': 1198,
 'similar': 5948}

In [ ]:
texts 

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel


In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel
xlmroberta = QuestionAnsweringModel(
    "bert", "/content/outputs/best_model")

In [ ]:
# โรงเรียนสวนกุหลาบวิทยาลัย (Suankularb Wittayalai School) (อักษรย่อ : ส.ก. / S.K.) 
# เป็นโรงเรียนชายล้วน ระดับชั้นมัธยมศึกษาขนาดใหญ่พิเศษ สังกัดสำนักงานเขตพื้นที่การศึกษามัธยมศึกษาเขต 1 สำนักงานคณะกรรมการการศึกษาขั้นพื้นฐาน
#  (ชื่อเดิม: กรมสามัญศึกษา) กระทรวงศึกษาธิการ ก่อตั้งโดย พระบาทสมเด็จพระจุลจอมเกล้าเจ้าอยู่หัว ได้รับการสถาปนาขึ้นในวันที่ 8 มีนาคม พ.ศ. 2424 
#  (ขณะนั้นนับวันที่ 1 เมษายน เป็นวันขึ้นปีใหม่ เมื่อนับอย่างสากลถือเป็น พ.ศ. 2425) โดยเป็นโรงเรียนรัฐบาลแห่งแรกของประเทศไทย

In [ ]:
question, text = "ดวงอาทิตย์มีมวลเท่าไหร่", "ดวงอาทิตย์ เป็นดาวฤกษ์ ณ ใจกลางระบบสุริยะ เป็นพลาสมาร้อนทรงเกือบกลมสมบูรณ์[4][5] โดยมีการเคลื่อนที่พาซึ่งผลิตสนามแม่เหล็กผ่านกระบวนการไดนาโม[6] ปัจจุบันเป็นแหล่งพลังงานสำคัญที่สุดสำหรับสิ่งมีชีวิตบนโลก มีเส้นผ่านศูนย์กลางประมาณ 1.39 ล้านกิโลเมตร ใหญ่กว่าโลก 109 เท่า และมีมวลประมาณ 330,000 เท่าของโลก คิดเป็นประมาณร้อยละ 99.86 ของมวลทั้งหมดของระบบสุริยะ[7] มวลประมาณสามในสี่ของดวงอาทิตย์เป็นไฮโดรเจน ส่วนที่เหลือเป็นฮีเลียมเป็นหลัก โดยมีปริมาณธาตุหนักกว่าเล็กน้อย รวมทั้งออกซิเจน คาร์บอน นีออนและเหล็ก[8]ดวงอาทิตย์เป็นดาวฤกษ์ลำดับหลักระดับจี (G2V) ตามการจัดประเภทดาวฤกษ์ตามระดับสเปกตรัม โดยมักถูกเรียกอย่างไม่เป็นทางการว่า  ดาวแคระเหลือง  ดวงอาทิตย์ก่อตัวขึ้นเมื่อประมาณ 4.6 พันล้านปีก่อน จากการยุบของแรงโน้มถ่วง (gravitational collapse) ของสสารภายในบริเวณกลุ่มเมฆโมเลกุลขนาดใหญ่ สสารนี้ส่วนใหญ่รวมอัดแน่นอยู่ที่ใจกลาง ส่วนที่เหลือบีบตัวลงลงเป็นแผ่นโคจรซึ่งกลายมาเป็นระบบสุริยะ มวลใจกลางร้อนและหนาแน่นมากจนเริ่มเกิดปฏิกิริยานิวเคลียร์ฟิวชั่น ณ แก่นดาว"
input = [
        {
            "context": text,
            "qas": [
                {
                    "question": question,
                    "id": "0",
                }
            ],
        }
    ]

In [ ]:
predictions, raw_outputs = bert.predict(input)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8612.53it/s]


Running Prediction:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(f'Ans :{predictions[0]["answer"][0]} prob : {raw_outputs[0]["probability"][0]}')

Ans : prob : 0.9999999999938856


In [ ]:
predictions

[{'answer': ['',
   '13,600,000 เคลวิน',
   '0.2 ถึง 0.7',
   '13,600,000',
   'อุณหภูมิประมาณ 13,600,000 เคลวิน',
   '2,000 กิโลเมตร',
   'เคลวิน',
   '0.2',
   'ภูมิประมาณ 13,600,000 เคลวิน',
   'หนาแน่นของน้ำบนโลก อุณหภูมิประมาณ 13,600,000 เคลวิน',
   'ิน',
   '13,600,000 เคลวิน ตลอดชีวิตส่วนใหญ่ของดวงอาทิตย์ ภายในแกนจะมีปฏิกิริยาฟิวชันลูกโซ่ โปรตอน-โปรตอน',
   'ประมาณ 13,600,000 เคลวิน',
   'ณหภูมิประมาณ 13,600,000 เคลวิน',
   'แน่นของน้ำบนโลก อุณหภูมิประมาณ 13,600,000 เคลวิน',
   'น่นของน้ำบนโลก อุณหภูมิประมาณ 13,600,000 เคลวิน',
   '0.7',
   'โอมกาก) ซึ่งอยู่ในช่วง 0.2 ถึง 0.7',
   'หนา 500 กิโลเมตร ชั้นถัดไปคือโครโมสเฟียร์ ซึ่งแปลว่ารงคมณฑล หรือทรงกลมแห่งสี เหตุที่เรียกชื่อนี้ก็เพราะเห็นเป็นแสงสีแวบขณะเกิดสุริยุปราคา ชั้นนี้หนา 2,000 กิโลเมตร'],
  'id': '0'}]

In [ ]:
raw_outputs

[{'id': '0',
  'probability': [0.9999993022540562,
   3.3563980522761246e-07,
   1.3092600636341974e-07,
   8.789956243568834e-08,
   4.0086458206797396e-08,
   1.4071670286559343e-08,
   1.4071670286559343e-08,
   1.1306904724760046e-08,
   9.823609064601006e-09,
   7.955401060722453e-09,
   7.771114393638568e-09,
   6.417363263380351e-09,
   5.619231027519011e-09,
   4.622253361171531e-09,
   4.497575962776382e-09,
   4.031601036934257e-09,
   3.938209099502371e-09,
   3.421575360358994e-09,
   3.1032341029794404e-09]}]

In [ ]:
!cp -r /content/outputs/best_model /content/drive/MyDrive/xlm-bestv2

^C


In [ ]:
# ความรัก เป็นความรู้สึก สภาพและเจตคติต่าง ๆ ซึ่งมีตั้งแต่ความชอบ ระหว่างบุคคลหมายถึงอารมณ์การดึงดูดและความผูกพัน (attachment)
#  ส่วนบุคคลอย่างแรงกล้า ในบริบททางปรัชญา ความรักเป็นคุณธรรมแสดงออกซึ่งความเมตตา ความเห็นอกเห็นใจ และความเสน่หาทั้งหมดของมนุษย์ 
#  ความรักเป็นแก่นของหลายศาสนา อย่างเช่นในวลี พระเจ้าเป็นความรัก ของศาสนาคริสต์ หรืออากาเปในพระวรสารในสารบบ 
#  ความรักยังอาจอธิบายได้ว่าเป็นพฤติกรรมต่อตนเองหรือผู้อื่นซึ่งตั้งอยู่บนพื้นฐานความเห็นอกเห็นใจ หรือความเสน่หา

In [ ]:
question, text = "", "ความรัก เป็นความรู้สึก สภาพและเจตคติต่าง ๆ ซึ่งมีตั้งแต่ความชอบ ระหว่างบุคคลหมายถึงอารมณ์การดึงดูดและความผูกพัน (attachment) ส่วนบุคคลอย่างแรงกล้า ในบริบททางปรัชญา ความรักเป็นคุณธรรมแสดงออกซึ่งความเมตตา ความเห็นอกเห็นใจ และความเสน่หาทั้งหมดของมนุษย์ ความรักเป็นแก่นของหลายศาสนา อย่างเช่นในวลี พระเจ้าเป็นความรัก ของศาสนาคริสต์ หรืออากาเปในพระวรสารในสารบบ ความรักยังอาจอธิบายได้ว่าเป็นพฤติกรรมต่อตนเองหรือผู้อื่นซึ่งตั้งอยู่บนพื้นฐานความเห็นอกเห็นใจ หรือความเสน่หา"
input = [
        {
            "context": text,
            "qas": [
                {
                    "question": question,
                    "id": "0",
                }
            ],
        }
    ]

In [ ]:
predictions, raw_outputs = Bert.predict(input)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6482.70it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions

[{'answer': ['',
   'พระเจ้าเป็นความรัก ของศาสนาคริสต์ หรืออากาเปในพระวรสารในสารบบ',
   'ของศาสนาคริสต์ หรืออากาเปในพระวรสารในสารบบ',
   'พระเจ้าเป็นความรัก ของศาสนาคริสต์',
   'หรืออากาเปในพระวรสารในสารบบ',
   'พระเจ้าเป็นความรัก ของศาสนาคริสต์ หรืออากาเปในพระวรสารในสารบบ ความรักยังอาจอธิบายได้ว่าเป็นพฤติกรรมต่อตนเองหรือผู้อื่นซึ่งตั้งอยู่บนพื้นฐานความเห็นอกเห็นใจ',
   'ของศาสนาคริสต์',
   'อย่างเช่นในวลี พระเจ้าเป็นความรัก ของศาสนาคริสต์ หรืออากาเปในพระวรสารในสารบบ',
   'พระเจ้าเป็นความรัก',
   'ความรักเป็นแก่นของหลายศาสนา อย่างเช่นในวลี พระเจ้าเป็นความรัก ของศาสนาคริสต์ หรืออากาเปในพระวรสารในสารบบ',
   'ความรักเป็นคุณธรรมแสดงออกซึ่งความเมตตา ความเห็นอกเห็นใจ และความเสน่หาทั้งหมดของมนุษย์ ความรักเป็นแก่นของหลายศาสนา อย่างเช่นในวลี พระเจ้าเป็นความรัก ของศาสนาคริสต์ หรืออากาเปในพระวรสารในสารบบ',
   'ของศาสนาคริสต์ หรืออากาเปในพระวรสารในสารบบ ความรักยังอาจอธิบายได้ว่าเป็นพฤติกรรมต่อตนเองหรือผู้อื่นซึ่งตั้งอยู่บนพื้นฐานความเห็นอกเห็นใจ',
   'อย่างเช่นในวลี พระเจ้าเป็นความรัก ของศาสนาคริส

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(f'Ans :{predictions[0]["answer"][1]} prob : {raw_outputs[0]["probability"][0]}')

Ans :เป็นความรู้สึก สภาพและเจตคติต่าง ๆ ซึ่งมีตั้งแต่ความชอบ prob : 0.5453349348676758


In [ ]:
!cp -r /content/outputs/best_model /content/drive/MyDrive/bert-model-last
